# LLMOps Workshop: 30-Minute Introduction

**Duration:** 30 minutes  
**Requirements:** Only local tools (no cloud services, no costs)

## What is LLMOps?

LLMOps (Large Language Model Operations) extends MLOps principles to manage the lifecycle of LLM applications:
- **Prompt Engineering & Versioning**
- **Model Evaluation & Testing**
- **Monitoring & Logging**
- **Deployment & Scaling**

## Workshop Overview
1. Setting up a local LLMOps environment
2. Prompt versioning and tracking
3. Simple evaluation metrics
4. Basic monitoring setup

## Part 1: Environment Setup (5 minutes)

We'll use only free, local tools:
- **Transformers** (Hugging Face) for local models
- **MLflow** for experiment tracking
- **JSON files** for simple logging

In [ ]:
"""
INSTALACIÓN DE DEPENDENCIAS PARA LLMOPS
=======================================

Este bloque instala todas las librerías necesarias para el workshop de LLMOps.

Librerías incluidas:
- transformers: Para cargar modelos de Hugging Face localmente
- torch: Motor de deep learning (PyTorch) requerido por transformers
- mlflow: Plataforma de tracking de experimentos de ML
- pandas: Manipulación y análisis de datos
- matplotlib: Visualización de datos
- seaborn: Gráficos estadísticos avanzados

Ejemplo de uso después de la instalación:
from transformers import pipeline
generator = pipeline('text-generation', model='gpt2')

NOTA: Ejecutar solo una vez al inicio del workshop
"""
# Instalar paquetes requeridos (ejecutar solo una vez)
# Comentar la línea siguiente después de la primera ejecución
!pip install transformers torch mlflow pandas matplotlib seaborn

In [ ]:
"""
CONFIGURACIÓN DEL ENTORNO LLMOPS LOCAL
=====================================

Este bloque configura el entorno base para nuestro pipeline de LLMOps.

Estructura de directorios creada:
📁 prompts/    -> Almacena versiones de prompts con metadatos
📁 logs/       -> Guarda métricas y resultados de evaluaciones  
📁 experiments/ -> Tracking de experimentos y configuraciones

Librerías importadas:
- json: Serialización de datos de prompts y métricas
- pandas: Análisis de datos de evaluación
- matplotlib: Visualización de métricas
- datetime: Timestamps para versionado
- transformers: Modelos de lenguaje locales
- mlflow: Tracking avanzado de experimentos
- os: Operaciones del sistema de archivos

Ejemplo de uso:
Los directorios permiten organizar:
- prompts/assistant_v1.json (versión 1 del prompt)
- logs/evaluation_metrics.json (métricas guardadas)
- experiments/model_comparison.json (resultados de experimentos)
"""

import json
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from transformers import pipeline
import mlflow
import os

# Crear estructura de directorios para nuestro setup de LLMOps
# exist_ok=True evita errores si los directorios ya existen
os.makedirs('prompts', exist_ok=True)     # Versionado de prompts
os.makedirs('logs', exist_ok=True)        # Métricas y logs
os.makedirs('experiments', exist_ok=True) # Tracking de experimentos

print("✅ Entorno LLMOps configurado correctamente!")

## Part 2: Load a Free Local Model (5 minutes)

We'll use a small, free model that runs locally without API costs.

In [ ]:
"""
CARGA DE MODELO LOCAL PARA LLMOPS
=================================

Este bloque carga un modelo de lenguaje que funciona completamente local,
sin costos de API ni conexión a internet requerida.

Modelo seleccionado: microsoft/DialoGPT-small
- Tamaño: ~117MB (modelo pequeño y rápido)
- Tipo: Generación de texto conversacional
- Ventajas: Gratuito, local, sin límites de tokens
- Desventajas: Menor calidad que modelos grandes (GPT-4, etc.)

Configuración del pipeline:
- task='text-generation': Tipo de tarea (generación de texto)
- return_full_text=False: Solo devuelve el texto generado (no el input)
- max_length=100: Máximo 100 tokens por respuesta

Ejemplo de uso después de cargar:
response = generator("Hola, ¿cómo estás?")
print(response[0]['generated_text'])

Alternativas de modelos locales:
- gpt2: Modelo base de OpenAI (más general)
- distilgpt2: Versión más rápida y ligera
- microsoft/DialoGPT-medium: Mejor calidad, más pesado
"""

# Cargar modelo de generación de texto local gratuito
print("Cargando modelo local DialoGPT-small...")
print("📁 Descargando ~117MB la primera vez...")

generator = pipeline(
    'text-generation',                    # Tarea: generación de texto
    model='microsoft/DialoGPT-small',    # Modelo conversacional pequeño
    return_full_text=False,              # Solo texto generado, no input
    max_length=100                       # Máximo 100 tokens por respuesta
)

print("✅ Modelo cargado exitosamente!")
print("🚀 Listo para generar texto localmente")

## Part 3: Prompt Versioning System (8 minutes)

Create a simple system to version and track prompts.

In [ ]:
"""
SISTEMA DE VERSIONADO DE PROMPTS PARA LLMOPS
===========================================

Esta clase implementa un sistema de control de versiones para prompts,
fundamental en LLMOps para rastrear la evolución y mejoras de prompts.

¿Por qué versionar prompts?
- Reproducibilidad: Poder recrear exactamente un experimento
- A/B Testing: Comparar diferentes versiones de prompts
- Rollback: Volver a versiones anteriores si hay problemas
- Auditoría: Rastrear qué cambios mejoraron el rendimiento

Estructura de datos guardada:
{
  "version": 1,
  "prompt": "Texto del prompt con {placeholders}",
  "description": "Descripción de los cambios",
  "created_at": "2024-01-15T10:30:00"
}

Ejemplo de evolución de prompts:
v1: "Responde sobre: {tema}"
v2: "Como experto, explica detalladamente: {tema}"
v3: "Eres un especialista en {área}. Explica {tema} con ejemplos prácticos"

Métodos de la clase:
- save_prompt(): Guarda nueva versión con metadatos automáticos
- load_prompt(): Carga versión específica o la más reciente
"""

class PromptManager:
    """
    Gestor de versiones de prompts para LLMOps
    
    Permite guardar, cargar y versionar prompts con metadatos automáticos.
    Cada prompt se guarda como JSON con timestamp y descripción.
    """
    
    def __init__(self, base_path='prompts'):
        """
        Inicializar el gestor de prompts
        
        Args:
            base_path (str): Directorio donde guardar los archivos de prompts
        """
        self.base_path = base_path
        self.current_version = 1
    
    def save_prompt(self, name, prompt, description=""):
        """
        Guardar una nueva versión de un prompt
        
        Args:
            name (str): Nombre identificador del prompt (ej: "assistant", "translator")
            prompt (str): Texto del prompt con placeholders opcionales {variable}
            description (str): Descripción de los cambios o propósito
            
        Returns:
            str: Ruta del archivo creado
            
        Ejemplo:
            pm.save_prompt("chatbot", "Responde como {persona}: {pregunta}", 
                          "Añadido contexto de personalidad")
        """
        prompt_data = {
            'version': self.current_version,           # Número de versión auto-incrementado
            'prompt': prompt,                          # Texto del prompt
            'description': description,                # Descripción de cambios
            'created_at': datetime.now().isoformat()  # Timestamp ISO 8601
        }
        
        # Crear nombre de archivo con versión: assistant_v1.json
        filename = f"{self.base_path}/{name}_v{self.current_version}.json"
        
        # Guardar prompt con formato JSON legible (indent=2)
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(prompt_data, f, indent=2, ensure_ascii=False)
        
        self.current_version += 1
        return filename
    
    def load_prompt(self, name, version=None):
        """
        Cargar un prompt específico por nombre y versión
        
        Args:
            name (str): Nombre del prompt a cargar
            version (int, optional): Versión específica. Si None, carga la más reciente
            
        Returns:
            dict: Datos del prompt incluyendo metadatos
            
        Ejemplo:
            prompt_data = pm.load_prompt("assistant", 1)
            texto = prompt_data['prompt'].format(tema="Python")
        """
        if version is None:
            version = self.current_version - 1  # Versión más reciente
        
        filename = f"{self.base_path}/{name}_v{version}.json"
        
        with open(filename, 'r', encoding='utf-8') as f:
            return json.load(f)

# Inicializar el gestor de prompts
pm = PromptManager()

# Crear ejemplos de prompts con evolución iterativa
prompt_v1 = "Generate a helpful response about: {topic}"
prompt_v2 = "As an expert assistant, provide detailed information about: {topic}"

# Guardar versiones con descripciones explicativas
pm.save_prompt("assistant", prompt_v1, "Prompt básico inicial")
pm.save_prompt("assistant", prompt_v2, "Mejorado con persona de experto")

print("✅ Sistema de versionado de prompts creado!")
print("📁 Archivos creados en directorio 'prompts/':")
print("   - assistant_v1.json (versión básica)")
print("   - assistant_v2.json (versión con persona)")

## Part 4: Simple Evaluation Framework (7 minutes)

Create basic metrics to evaluate LLM responses.

In [ ]:
"""
FRAMEWORK DE EVALUACIÓN PARA LLMOPS
==================================

Esta clase implementa un sistema de evaluación de respuestas de LLMs,
crucial para medir y mejorar la calidad de las respuestas generadas.

Métricas implementadas:
1. response_length: Longitud en caracteres de la respuesta
2. word_count: Número de palabras en la respuesta  
3. keyword_score: Porcentaje de palabras clave encontradas (0.0 - 1.0)

¿Por qué evaluar respuestas LLM?
- Calidad: Medir si las respuestas son apropiadas
- Consistencia: Verificar comportamiento estable entre versiones
- Mejora continua: Identificar áreas de optimización
- A/B Testing: Comparar diferentes prompts objetivamente

Estructura de métricas guardadas:
{
  "timestamp": "2024-01-15T10:30:00",
  "prompt": "Prompt usado",
  "response": "Respuesta generada",
  "response_length": 150,
  "word_count": 25,
  "keyword_score": 0.75,
  "expected_keywords": ["algoritmo", "datos", "modelo"]
}

Ejemplo de evaluación:
evaluator.evaluate_response(
    prompt="Explica machine learning",
    response="Machine learning usa algoritmos para analizar datos",
    expected_keywords=["algoritmo", "datos", "modelo"]
)
# Resultado: keyword_score = 0.67 (2 de 3 palabras encontradas)
"""

class LLMEvaluator:
    """
    Sistema de evaluación de respuestas de modelos de lenguaje
    
    Calcula métricas automáticas para evaluar calidad y relevancia
    de las respuestas generadas por LLMs.
    """
    
    def __init__(self):
        """Inicializar evaluador con lista vacía de métricas"""
        self.metrics = []
    
    def evaluate_response(self, prompt, response, expected_keywords=None):
        """
        Evaluar una respuesta del LLM con múltiples métricas
        
        Args:
            prompt (str): Prompt usado para generar la respuesta
            response (str): Respuesta generada por el modelo
            expected_keywords (list, optional): Palabras clave esperadas
            
        Returns:
            dict: Diccionario con todas las métricas calculadas
            
        Ejemplo:
            metrics = evaluator.evaluate_response(
                prompt="¿Qué es Python?",
                response="Python es un lenguaje de programación fácil de aprender",
                expected_keywords=["lenguaje", "programación", "código"]
            )
            print(f"Score: {metrics['keyword_score']}")  # 0.67
        """
        
        # Métricas básicas de longitud
        response_length = len(response)                    # Caracteres totales
        word_count = len(response.split())                # Palabras totales
        
        # Métrica de cobertura de palabras clave
        keyword_score = 0
        if expected_keywords:
            # Contar cuántas keywords aparecen en la respuesta (case-insensitive)
            found_keywords = sum(
                1 for kw in expected_keywords 
                if kw.lower() in response.lower()
            )
            # Calcular porcentaje de cobertura (0.0 a 1.0)
            keyword_score = found_keywords / len(expected_keywords)
        
        # Estructura completa de métricas con metadatos
        metrics = {
            'timestamp': datetime.now().isoformat(),       # Cuándo se evaluó
            'prompt': prompt,                              # Prompt original
            'response': response,                          # Respuesta evaluada
            'response_length': response_length,            # Métrica: longitud
            'word_count': word_count,                      # Métrica: palabras
            'keyword_score': keyword_score,                # Métrica: relevancia
            'expected_keywords': expected_keywords or []   # Keywords esperadas
        }
        
        # Agregar a la lista de métricas para análisis posterior
        self.metrics.append(metrics)
        return metrics
    
    def save_metrics(self, filename='logs/evaluation_metrics.json'):
        """
        Guardar todas las métricas en archivo JSON
        
        Args:
            filename (str): Ruta donde guardar las métricas
            
        Ejemplo:
            evaluator.save_metrics('experimento_01_metrics.json')
        """
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(self.metrics, f, indent=2, ensure_ascii=False)
    
    def get_summary(self):
        """
        Obtener resumen estadístico de todas las métricas
        
        Returns:
            pandas.DataFrame: Estadísticas descriptivas (mean, std, min, max, etc.)
            
        Ejemplo:
            summary = evaluator.get_summary()
            print(f"Longitud promedio: {summary.loc['mean', 'response_length']}")
        """
        if not self.metrics:
            return "No hay métricas disponibles para analizar"
        
        # Convertir métricas a DataFrame para análisis estadístico
        df = pd.DataFrame(self.metrics)
        
        # Seleccionar solo columnas numéricas para estadísticas
        numeric_columns = ['response_length', 'word_count', 'keyword_score']
        return df[numeric_columns].describe()

# Inicializar el evaluador
evaluator = LLMEvaluator()

print("✅ Framework de evaluación configurado!")
print("📊 Métricas disponibles:")
print("   - response_length: Longitud en caracteres")
print("   - word_count: Número de palabras")
print("   - keyword_score: Cobertura de palabras clave (0.0-1.0)")

## Part 5: Hands-on Example (5 minutes)

Let's test our LLMOps pipeline with real examples.

In [ ]:
"""
EJEMPLO PRÁCTICO: PIPELINE COMPLETO DE LLMOPS
=============================================

Este bloque demuestra un pipeline completo de LLMOps ejecutando:
1. Carga de múltiples versiones de prompts
2. Generación de respuestas con modelo local
3. Evaluación comparativa de respuestas
4. Almacenamiento de métricas para análisis

Flujo del experimento:
📝 Cargar prompts v1 y v2
🎯 Formatear con temas de prueba  
🤖 Generar respuestas con modelo local
📊 Evaluar calidad con métricas automáticas
💾 Guardar resultados para tracking

Temas de prueba seleccionados:
- "machine learning": Algoritmos de aprendizaje automático
- "data science": Ciencia de datos y análisis
- "artificial intelligence": Inteligencia artificial

Palabras clave para evaluación:
- "algorithm": Debe aparecer en respuestas técnicas
- "data": Fundamental en temas de datos
- "model": Concepto clave en ML/AI
- "prediction": Objetivo común en estos campos

Comparación A/B Testing:
- V1: Prompt básico sin contexto específico
- V2: Prompt con persona de "experto" para mejores respuestas
"""

# Configuración del experimento de evaluación comparativa
test_topics = ["machine learning", "data science", "artificial intelligence"]
test_keywords = ["algorithm", "data", "model", "prediction"]

print("🧪 Iniciando experimento de evaluación comparativa")
print("📋 Temas a evaluar:", test_topics)
print("🔍 Palabras clave esperadas:", test_keywords)
print("\n" + "="*60)

# Iterar sobre cada tema para evaluación completa
for i, topic in enumerate(test_topics, 1):
    print(f"\n📊 Experimento {i}/3 - Tema: '{topic}'")
    
    # PASO 1: Cargar ambas versiones de prompts desde archivos JSON
    prompt_v1_data = pm.load_prompt("assistant", 1)  # Versión básica
    prompt_v2_data = pm.load_prompt("assistant", 2)  # Versión con persona
    
    # PASO 2: Formatear prompts con el tema actual usando placeholders
    formatted_prompt_v1 = prompt_v1_data['prompt'].format(topic=topic)
    formatted_prompt_v2 = prompt_v2_data['prompt'].format(topic=topic)
    
    print(f"   🔄 Prompts formateados para '{topic}'")
    
    # PASO 3: Generar respuestas usando el modelo local
    try:
        # Intentar generación real con el modelo DialoGPT
        response_v1 = generator(formatted_prompt_v1, max_length=50, num_return_sequences=1)[0]['generated_text']
        response_v2 = generator(formatted_prompt_v2, max_length=50, num_return_sequences=1)[0]['generated_text']
        print(f"   ✅ Respuestas generadas por modelo local")
        
    except Exception as e:
        # Fallback con respuestas simuladas si hay errores
        print(f"   ⚠️  Modelo no disponible, usando respuestas simuladas")
        response_v1 = f"Basic information about {topic} including algorithms and data processing."
        response_v2 = f"As an expert, {topic} involves sophisticated algorithms, data analysis, and predictive modeling techniques."
    
    # PASO 4: Evaluar ambas respuestas con métricas automáticas
    eval_v1 = evaluator.evaluate_response(
        prompt=formatted_prompt_v1,
        response=response_v1,
        expected_keywords=test_keywords
    )
    
    eval_v2 = evaluator.evaluate_response(
        prompt=formatted_prompt_v2,
        response=response_v2,
        expected_keywords=test_keywords
    )
    
    # PASO 5: Mostrar comparación de resultados
    print(f"   📈 Resultados comparativos:")
    print(f"      V1 (básico)  - Longitud: {eval_v1['response_length']:3d} chars, Keywords: {eval_v1['keyword_score']:.2f}")
    print(f"      V2 (experto) - Longitud: {eval_v2['response_length']:3d} chars, Keywords: {eval_v2['keyword_score']:.2f}")
    
    # Determinar qué versión tuvo mejor rendimiento
    if eval_v2['keyword_score'] > eval_v1['keyword_score']:
        print(f"      🏆 V2 (experto) obtuvo mejor keyword score")
    elif eval_v1['keyword_score'] > eval_v2['keyword_score']:
        print(f"      🏆 V1 (básico) obtuvo mejor keyword score")
    else:
        print(f"      🤝 Ambas versiones obtuvieron el mismo keyword score")

print("\n" + "="*60)

# PASO 6: Guardar todas las métricas para análisis posterior
evaluator.save_metrics()
print("✅ Experimento completado!")
print("💾 Métricas guardadas en 'logs/evaluation_metrics.json'")
print("📊 Datos listos para análisis y dashboard de monitoreo")

## Part 6: Simple Monitoring Dashboard (Bonus)

Create basic visualizations of our metrics.

In [ ]:
"""
DASHBOARD DE MONITOREO PARA LLMOPS
=================================

Este bloque crea visualizaciones para monitorear el rendimiento del LLM,
elemento esencial en LLMOps para tracking continuo de la calidad.

Gráficos generados:
1. Distribución de longitud de respuestas
   - Identifica respuestas muy cortas (posibles errores)
   - Detecta respuestas muy largas (posible divagación)
   - Muestra consistencia en la longitud de output

2. Distribución de keyword score
   - Mide relevancia de respuestas (0.0 = irrelevante, 1.0 = perfecta)
   - Identifica problemas de calidad en prompts
   - Compara rendimiento entre versiones

¿Por qué monitorear LLMs?
- Detectar degradación de calidad en tiempo real
- Identificar prompts problemáticos
- Validar mejoras en nuevas versiones
- Mantener SLAs de calidad en producción

Métricas adicionales para producción:
- Tiempo de respuesta (latencia)
- Tasa de errores del modelo
- Costos de tokens (si usa APIs de pago)
- Satisfacción del usuario (feedback)

Alertas recomendadas:
- Keyword score < 0.3 (baja relevancia)
- Response length < 10 chars (respuestas muy cortas)
- Response length > 1000 chars (posible alucinación)
"""

# Verificar si tenemos métricas para visualizar
if evaluator.metrics:
    print("📊 Generando dashboard de monitoreo...")
    
    # Convertir métricas a DataFrame para análisis
    df = pd.DataFrame(evaluator.metrics)
    print(f"📈 Procesando {len(df)} evaluaciones realizadas")
    
    # Crear figura con 2 subplots lado a lado
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    fig.suptitle('Dashboard de Monitoreo LLMOps', fontsize=16, fontweight='bold')
    
    # GRÁFICO 1: Distribución de longitud de respuestas
    axes[0].hist(df['response_length'], bins=10, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0].set_title('Distribución de Longitud de Respuestas', fontweight='bold')
    axes[0].set_xlabel('Caracteres')
    axes[0].set_ylabel('Frecuencia')
    axes[0].grid(True, alpha=0.3)
    
    # Agregar líneas de referencia para alertas
    mean_length = df['response_length'].mean()
    axes[0].axvline(mean_length, color='red', linestyle='--', 
                   label=f'Promedio: {mean_length:.0f}')
    axes[0].legend()
    
    # GRÁFICO 2: Distribución de keyword score
    axes[1].hist(df['keyword_score'], bins=10, alpha=0.7, color='lightgreen', edgecolor='black')
    axes[1].set_title('Distribución de Keyword Score', fontweight='bold')
    axes[1].set_xlabel('Score (0.0 - 1.0)')
    axes[1].set_ylabel('Frecuencia')
    axes[1].grid(True, alpha=0.3)
    
    # Agregar líneas de referencia para calidad
    mean_score = df['keyword_score'].mean()
    axes[1].axvline(mean_score, color='red', linestyle='--', 
                   label=f'Promedio: {mean_score:.2f}')
    axes[1].axvline(0.5, color='orange', linestyle=':', 
                   label='Umbral mínimo: 0.5')
    axes[1].legend()
    
    # Ajustar layout y guardar dashboard
    plt.tight_layout()
    plt.savefig('logs/monitoring_dashboard.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Dashboard guardado en 'logs/monitoring_dashboard.png'")
    
    # ANÁLISIS ESTADÍSTICO DETALLADO
    print("\n📈 Resumen Estadístico Completo:")
    print("="*50)
    
    # Obtener estadísticas descriptivas
    summary = evaluator.get_summary()
    print(summary)
    
    # ALERTAS AUTOMÁTICAS basadas en umbrales
    print("\n🚨 Sistema de Alertas:")
    print("="*30)
    
    # Verificar métricas problemáticas
    low_quality_responses = df[df['keyword_score'] < 0.3]
    very_short_responses = df[df['response_length'] < 10]
    very_long_responses = df[df['response_length'] > 500]
    
    if len(low_quality_responses) > 0:
        print(f"⚠️  ALERTA: {len(low_quality_responses)} respuestas con baja relevancia (score < 0.3)")
    
    if len(very_short_responses) > 0:
        print(f"⚠️  ALERTA: {len(very_short_responses)} respuestas muy cortas (< 10 caracteres)")
    
    if len(very_long_responses) > 0:
        print(f"⚠️  ALERTA: {len(very_long_responses)} respuestas muy largas (> 500 caracteres)")
    
    if len(low_quality_responses) == 0 and len(very_short_responses) == 0 and len(very_long_responses) == 0:
        print("✅ Todas las métricas están dentro de rangos aceptables")
    
    # RECOMENDACIONES AUTOMÁTICAS
    avg_score = df['keyword_score'].mean()
    if avg_score < 0.5:
        print(f"\n💡 Recomendación: Keyword score promedio ({avg_score:.2f}) es bajo.")
        print("   Considera mejorar los prompts o ajustar las palabras clave esperadas.")
    elif avg_score > 0.8:
        print(f"\n🎉 Excelente: Keyword score promedio ({avg_score:.2f}) es muy bueno.")
    
else:
    print("📊 No hay métricas disponibles para el dashboard.")
    print("   Ejecuta primero el bloque de evaluación para generar datos.")

## Summary: What We Built (Bonus)

In 30 minutes, we created a complete local LLMOps pipeline:

### ✅ Components Created:
1. **Prompt Versioning System** - Track and manage prompt evolution
2. **Local Model Integration** - No API costs, runs offline
3. **Evaluation Framework** - Measure response quality
4. **Simple Monitoring** - Basic metrics and visualizations
5. **Experiment Tracking** - Log all interactions and results

### 🎯 Key LLMOps Principles Covered:
- **Reproducibility** - Version control for prompts
- **Evaluation** - Systematic quality measurement
- **Monitoring** - Track performance over time
- **Cost Management** - Use free, local resources

### 🚀 Next Steps:
- Integrate with MLflow for advanced experiment tracking
- Add more sophisticated evaluation metrics
- Implement automated testing pipelines
- Scale to production deployment